In [1]:
import numpy as np
import scipy.signal as sps
import sympy as sym 
import scipy.optimize as sopt
%matplotlib inline   
import matplotlib.pyplot as plt

In [3]:
eta_1_d,eta_1_q = sym.symbols('eta_1_d,eta_1_q')
i_1_d,i_1_q,v_1_dc,i_1_dc = sym.symbols('i_1_d,i_1_q,v_1_dc,i_1_dc')
v_1_sd,v_1_sq = sym.symbols('v_1_sd,v_1_sq')
L_1,R_1,C_1,omega_1 = sym.symbols('L_1,R_1,C_1,omega_1')

R_dc = sym.symbols('R_dc')

v_th_d,v_th_q = sym.symbols('v_th_d,v_th_q')
R_th,L_th = sym.symbols('R_th,L_th')
R_12,L_12 = sym.symbols('R_12,L_12')

p_1,q_1 = sym.symbols('p_1,q_1')

In [4]:
v_1_sd = R_th*(i_1_d  + i_2_d) + v_th_d
v_1_sq = R_th*(i_1_q  + i_2_q) + v_th_q

di_1_d  = 1/L_1*(eta_1_d*v_1_dc/2 - R_1*i_1_d - L_1*omega_1*i_1_q - v_1_sd)
di_1_q  = 1/L_1*(eta_1_q*v_1_dc/2 - R_1*i_1_q + L_1*omega_1*i_1_d - v_1_sq)

i_dc = (v_1_dc -v_2_dc)/R_dc

# p_dc = p_ac = 3/2*(eta_d*v_dc/2*i_d + eta_q*v_dc/2*i_q)
# i_cap = i_dc - p_dc/v_dc
# i_dc - 3/4*(eta_d*i_d + eta_q*i_q)
dv_1_dc = 1/C_1*(-i_dc - 3/4*(eta_1_d*i_1_d + eta_1_q*i_1_q ))

v_2_sd = R_12*(i_2_d) + v_1_sd
v_2_sq = R_12*(i_2_q) + v_1_sq

di_2_d  = 1/L_2*(eta_2_d*v_2_dc/2 - R_2*i_2_d - L_2*omega_2*i_2_q - v_2_sd)
di_2_q  = 1/L_2*(eta_2_q*v_2_dc/2 - R_2*i_2_q + L_2*omega_2*i_2_d - v_2_sq)

# p_dc = p_ac = 3/2*(eta_d*v_dc/2*i_d + eta_q*v_dc/2*i_q)
# i_cap = i_dc - p_dc/v_dc
# i_dc - 3/4*(eta_d*i_d + eta_q*i_q)
dv_2_dc = 1/C_2*( i_dc - 3/4*(eta_2_d*i_2_d + eta_2_q*i_2_q ))


f = sym.Matrix([[di_1_d],[di_1_q],[dv_1_dc], [di_2_d],[di_2_q],[dv_2_dc]])
x = sym.Matrix([[i_1_d],[i_1_q],[v_1_dc],[i_2_d],[i_2_q],[v_2_dc]])

F_x = f.jacobian(x)

F_x

NameError: name 'i_2_d' is not defined

## $\sf i_d$-$\sf i_q$ mode

In [ ]:
params = {'v_th_q':np.sqrt(2./3)*400.0,
          'v_th_d':0.0,
          'R_th':0.001,
          'R_dc':0.02,
          'R_12':0.001}

params_1 = {'L_1':2.5e-3,
            'R_1':1,
            'C_1':2000.0e-6,
            'omega_1':2.0*np.pi*50,
            'i_1_d':0.0,
            'v_1_dc':800.0
           }

params.update(params_1)

i_2_q_N = -20.0
params_2 = {'L_2':2.5e-3,
            'R_2':1.0,
            'C_2':2000.0e-6,
            'omega_2':2.0*np.pi*50,
            'i_2_d':5.0,
            'i_2_q':i_2_q_N,
           }

params.update(params_2)

f_ss_sym = f.subs(params)
sym.pretty_print(f_ss_sym)

#u = sym.Matrix([i_1_q,eta_1_d,eta_1_q,v_1_sd,v_1_sq,eta_2_d,eta_2_q,v_2_sd,v_2_sq,v_2_dc])
u = sym.Matrix([eta_1_d,eta_1_q,i_1_q,eta_2_d,eta_2_q,v_2_dc])

f_ss = sym.lambdify(u, f_ss_sym,'numpy') # returns a numpy-ready function

def eqs_ss(x):
    
    eta_1_d,eta_1_q,i_1_q  = x[0],x[1],x[2]
    eta_2_d,eta_2_q,v_2_dc = x[3],x[4],x[5]

    
    return f_ss(eta_1_d,eta_1_q,i_1_q,eta_2_d,eta_2_q,v_2_dc )

sol = sopt.anderson(eqs_ss, [0.7,0,-i_2_q_N,0,0,890], f_tol=1e-6, iter = 5000)

print(sol)
print('eta_1_d = {:2.3f}, eta_1_q = {:2.3f}'.format(sol[0],sol[1]))
print('eta_2_d = {:2.3f}, eta_2_q = {:2.3f}'.format(sol[3],sol[4]))

params_s = {'eta_1_d':sol[0],
            'eta_1_q':sol[1],
            'i_1_q':sol[2],
            'eta_2_d':sol[3],
            'eta_2_q':sol[4],
            'v_2_dc':sol[5],
           }

params.update(params_s)

print(params_s)

i_dc = (v_1_dc -v_2_dc)/R_dc

sym.pretty_print(i_dc.subs(params))

## $\sf p$-$\sf q$ mode

In [ ]:
v_1_dc_N = 900
p_2_N = 10.0e3

R_dc_N = 0.001

params = {'v_th_q':np.sqrt(2./3)*400.0,
          'v_th_d':0.0,
          'R_th':0.001,
          'R_dc':R_dc_N,
          'R_12':0.001}



params_1 = {'L_1':2.5e-3,
            'R_1':0.5,
            'C_1':2000.0e-6,
            'omega_1':2.0*np.pi*50,
            'q_1':0.0e3,
            'v_1_dc':v_1_dc_N
           }

params.update(params_1)


params_2 = {'L_2':2.5e-3,
            'R_2':0.5,
            'C_2':2000.0e-6,
            'omega_2':2.0*np.pi*50,
            'p_2':p_2_N,
            'q_2':0.0,
           }

g_1_p = 3/2*(i_1_d*v_1_sd + i_1_q*v_1_sq ) - p_1
g_1_q = 3/2*(i_1_q*v_1_sd - i_1_d*v_1_sq ) - q_1
g_2_p = 3/2*(i_2_d*v_2_sd + i_2_q*v_2_sq ) - p_2
g_2_q = 3/2*(i_2_q*v_2_sd - i_2_d*v_2_sq ) - q_2


g = sym.Matrix([[g_1_p],[g_1_q],[g_2_p], [g_2_q]])
y = sym.Matrix([[p_1],[q_1],[p_2],[q_2]])


params.update(params_2)


fg = f.row_insert(6,g)
fg_ss_sym = fg.subs(params)
#sym.pretty_print(fg_ss_sym)

#u = sym.Matrix([i_1_q,eta_1_d,eta_1_q,v_1_sd,v_1_sq,eta_2_d,eta_2_q,v_2_sd,v_2_sq,v_2_dc])
u = sym.Matrix([eta_1_d,eta_1_q,i_1_d,i_1_q,p_1,eta_2_d,eta_2_q,i_2_d,i_2_q,v_2_dc])

fg_ss = sym.lambdify(u, fg_ss_sym,'numpy') # returns a numpy-ready function

def eqs_ss(x):
    
    eta_1_d,eta_1_q,i_1_d,i_1_q,p_1  = x[0],x[1],x[2],x[3],x[4]
    eta_2_d,eta_2_q,i_2_d,i_2_q,v_2_dc = x[5],x[6],x[7],x[8],x[9]

    
    return fg_ss(eta_1_d,eta_1_q,i_1_d,i_1_q,p_1,eta_2_d,eta_2_q,i_2_d,i_2_q,v_2_dc).reshape(10,)


v_2_dc_0 = v_1_dc_N - p_2_N/v_1_dc_N*R_dc_N

sol_obj = sopt.root(eqs_ss, [0.0,0.72,0.0,-p_2_N/325*2/3,-p_2_N,0.0,0.72,0.0,p_2_N/325*2/3,v_2_dc_0],method='hybr')

print(sol_obj.success)
sol = sol_obj.x

params_s = {'eta_1_d':sol[0],
            'eta_1_q':sol[1],
            'i_1_d':sol[2],
            'i_1_q':sol[3],
            'p_1':sol[4],
            'eta_2_d':sol[5],
            'eta_2_q':sol[6],
            'i_2_d':sol[7],
            'i_2_q':sol[8],            
            'v_2_dc':sol[9],
           }

params.update(params_s)

print(params_s)

i_dc = (v_1_dc -v_2_dc)/R_dc

sym.pretty_print(i_dc.subs(params))

In [ ]:
sol.x

## $\sf i_d$-$\sf v_{dc}$ mode

In [ ]:
%matplotlib inline   
import matplotlib.pyplot as plt

fig_bodes = plt.figure(figsize=(10,8))

ax_rlocus     = fig_bodes.add_subplot(1,1,1)

ax_rlocus.plot(eig.real,eig.imag, 'o')

ax_rlocus.grid(True)

plt.show()

In [ ]:
G = sps.ss2tf(A,B,C,D)
G

In [ ]:
H_x

In [ ]:
np.linalg.eig(A)

In [ ]:
C

In [ ]:
from sympy.utilities.codegen import codegen



In [ ]:
codegen(("f", f),'c')

In [ ]:
f

In [ ]:
L = params_1['L_1']
C = params_1['C_1']
Vs = 400.0
Ps0 = 100e3
tau = 2*L*Ps0/(3*Vs**2)
Gv = sps.lti([tau,1],[-C/2*1, 0])
tv,yv = sps.step(Gv)
Gint = sps.lti([-2/C],[1, 0])
ti,yi = sps.step(Gint)

fig = plt.figure(figsize=(10,8))
ax= fig.add_subplot(1,1,1)
#ax.plot(ti,yi)
#ax.plot(tv,yv)
ax.plot(ti,yi-yv)


ax.grid(True)

plt.show()


In [ ]:
tau